# **CREATING THE FINAL PANEL**

In [1]:
import param
import panel as pn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from bokeh.plotting import figure
import networkx as nx
import matplotlib.pyplot as plt
from math import pi
from bokeh.palettes import Category20c, Category20, Category10
from bokeh.plotting import figure
from bokeh.transform import cumsum
import numpy as np
import io
import pickle
import torch

In [3]:
#path = 'C:/Users/Usuario/Desktop/UAB/Tercer/2n cuatri/Synthesis project/GraphAnomaly/elliptic_bitcoin_dataset/'
path = 'C:/Users/User/Desktop/Assignatures/Synthesis project/GraphAnomaly/dades_arnau/'
with open(path + 'sequential_train.pkl', 'rb') as f:
    sub_train = pickle.load(f)
with open(path + 'sequential_test.pkl', 'rb') as f:
    sub_test = pickle.load(f)
with open(path + 'edges.pkl', 'rb') as f:
    df_edges_init = pickle.load(f)

In [4]:
# Obtain trained random forest model
def train_random_forest(train_set, test_set):
    X_train = train_set.loc[train_set['class'].isin([0, 1])].drop(columns=['class'])
    y_train = list(train_set.loc[train_set['class'].isin([0, 1])]['class'])
    
    X_test = test_set.loc[test_set['class'].isin([0, 1])].drop(columns=['class'])
    y_test = list(test_set.loc[test_set['class'].isin([0, 1])]['class'])
    
    model = RandomForestClassifier()

    return model.fit(X_train, y_train)

random_forest_model = train_random_forest(sub_train, sub_test)

In [6]:
class dashboard:
    def __init__(self, test_set, df_edges_init, random_forest_model):
        self.test_set = test_set
        self.df_edges_init = df_edges_init
        self.df_numerical_results = pd.DataFrame(columns=["Licit", "Ilicit"])
        self.rf_model = random_forest_model      

    def load_file(self):
        if file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded file {self.loaded_file}")
        else: # local copy
            self.loaded_file = file_input
            button_load.button_style="outline"
            self.df = pd.read_csv(io.BytesIO(self.loaded_file.value))            
            button_load.button_style="solid"
        return self.df
        
        
    def display_df_predictions(self, timestep, model):
        # Create a subdataframe based on the timestep
        self.df_subgraph = self.test_set[self.test_set['time step'] == timestep]
        self.df_edges = self.df_edges_init.loc[(self.df_edges_init['txId1'].isin(self.df_subgraph['node'])) & (self.df_edges_init['txId2'].isin(self.df_subgraph['node']))]
        
        # Obtain the predictions from the trained model chosen by the user
        X_test = self.df_subgraph.drop(columns=['class'])
        
        if model == "Random Forest":
            self.y_pred = self.rf_model.predict(X_test)
            
        elif model == "GNN":
            # self.y_pred = self.GNN_model.predict(X_test)
            pass
            
        elif model == "Autoencoders Embeddings":
            pass
            
        else:
            pass


        # Store the predictions of the model in a dataframe
        # Only display the id node, time step and the class
        self.df_predictions = self.df_subgraph.iloc[:, 0:3]
        # Rename some columns
        columns_to_replace = {'node': 'Id Node', 'class': 'True Class', 'time step': 'Time Step'}
        self.df_predictions.rename(columns=columns_to_replace, inplace=True)
        # Insert the predictions
        self.df_predictions.insert(loc=2, column='Predicted Class', value=self.y_pred)
        # Replace the numbers for licit, ilicit or unknown
        df_final_predictions = self.df_predictions.replace({0: "Ilicit", 1: "Licit", 2: "Unknown"})

        return df_final_predictions


    def summary_results(self):
        # Create a dataframe of number of licit and ilicit nodes of predicted and true labels
        licit = {
            "True Label": self.df_predictions['True Class'].value_counts()[1],
            "Predicted Label": self.df_predictions['Predicted Class'].value_counts()[1]
        }
        
        ilicit = {
            "True Label": self.df_predictions['True Class'].value_counts()[0],
            "Predicted Label": self.df_predictions['Predicted Class'].value_counts()[0]
        }
        
        summary = {
            "Licit": licit,
            "Ilicit": ilicit
        }

        self.df_summary_results = pd.DataFrame(summary)

        return pn.widgets.Tabulator(self.df_summary_results)


    def display_pie_charts(self, label):
        if self.df_summary_results is None:
            self.summary_results()
        # Pie chart for true labels
        if label=="true":
            labels = self.df_summary_results.loc["True Label"]
            data = labels.reset_index(name='value').rename(columns={'index':'TrueLabel'})
            
            # Pie chart
            data['angle'] = data['value']/data['value'].sum() * 2*pi
            data['color'] = Category10[10][2:4]
            
            p = figure(height=350, title="True Labels", toolbar_location=None,
                       tools="hover", tooltips="@TrueLabel: @value", x_range=(-0.5, 1.0))
            
            r = p.wedge(x=0, y=1, radius=0.4,
                    start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                    line_color="white", fill_color='color', legend_field='TrueLabel', source=data)
            
            p.axis.axis_label=None
            p.axis.visible=False
            p.grid.grid_line_color = None


        # Pie chart for predicted label
        else:
            labels = self.df_summary_results.loc["Predicted Label"]
            data = labels.reset_index(name='value').rename(columns={'index':'PredictedLabel'})

            # Pie chart
            data['angle'] = data['value']/data['value'].sum() * 2*pi
            data['color'] = Category10[10][2:4]
            
            p = figure(height=350, title="Predicted Labels", toolbar_location=None,
                       tools="hover", tooltips="@PredictedLabel: @value", x_range=(-0.5, 1.0))
            
            r = p.wedge(x=0, y=1, radius=0.4,
                    start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                    line_color="white", fill_color='color', legend_field='PredictedLabel', source=data)
            
            p.axis.axis_label=None
            p.axis.visible=False
            p.grid.grid_line_color = None

        
        return pn.pane.Bokeh(p, theme="dark_minimal")
        
    
    def display_graph(self, label):
        # Create graph
        graph = nx.Graph()
        
        for _, row in self.df_predictions.iterrows():
            # Extract node ID and attributes
            node_id = row['Id Node']
            node_attributes = row.drop('Id Node').to_dict()
            
            # Add node to the graph with its attributes
            graph.add_node(node_id, **node_attributes)

        # Add edges
        for _, row in self.df_edges.iterrows():
            graph.add_edge(row['txId1'], row['txId2'])

        
        # Display graph
        fig,ax=plt.subplots()
        pos = nx.spring_layout(graph)
        
        if label=="true": # True graph
            colors = {0: 'red', 1: 'green', 2: 'black'}
            nx.draw(graph, pos, ax=ax,with_labels=False, node_color=[colors[graph.nodes[n]['True Class']] for n in graph.nodes()], node_size=20, font_size=12)
        else: # Predicted graph
            colors = {0: 'red', 1: 'green'}
            nx.draw(graph, pos, ax=ax,with_labels=False, node_color=[colors[graph.nodes[n]['Predicted Class']] for n in graph.nodes()], node_size=20, font_size=12)
        
        return fig



db = dashboard(sub_test, df_edges_init, random_forest_model)

In [9]:
pn.extension('tabulator', design='material', loading_spinner='dots', loading_color='#00aa41', template='bootstrap', notifications=True)
pn.widgets.Tabulator.theme = 'materialize'
pn.param.ParamMethod.loading_indicator = True

#*****************************************************************************************************
# For tab 1: create buttons to enter the text and upload the file, also a button to clear the history
#*****************************************************************************************************
file_input = pn.widgets.FileInput()
button_load = pn.widgets.Button(name="Load", button_type='primary')

load_df = pn.bind(db.load_file)

def result(clicked):
    if clicked:
        return load_df()

loaded_file = pn.bind(result, button_load)

tab1 = pn.Column(
    pn.Row( file_input, button_load),
    pn.widgets.Tabulator(loaded_file)
)

#********************************************************************************************
# Tab 2: Creating options for the user to choose from and displaying the predictions obtained
#********************************************************************************************
timestep_widget = pn.widgets.IntSlider(name="TimeStep", value=35, start=35, end=48)
model_widget = pn.widgets.RadioButtonGroup(name="Models", options=["Random Forest", "GNN", "Autoencoders Embeddings", "Autoencoders Reconstruction"])

predictions = pn.bind(db.display_df_predictions, timestep=timestep_widget, model=model_widget)

button_get_predictions = pn.widgets.Button(name="Get predictions", button_type="primary")

configuration_box = pn.WidgetBox('## Select configuration', timestep_widget, model_widget, button_get_predictions)

def result(clicked):
    if clicked:
        return predictions()


df_predictions = pn.bind(result, button_get_predictions)


# Add filters to the df_predictions
filters = {
    'True Class': {'type': 'list', 'func': 'in', 'valuesLookup': True, 'sort': 'asc', 'multiselect': True},
    'Predicted Class': {'type': 'list', 'func': 'in', 'valuesLookup': True, 'sort': 'asc', 'multiselect': True},
}

filter_table = pn.widgets.Tabulator(
    df_predictions, pagination='local', layout='fit_columns', page_size=10, sizing_mode='stretch_width',
    header_filters=filters)

filename, button_download = filter_table.download_menu(
    text_kwargs={'name': 'Enter filename', 'value': 'predictions.csv'},
    button_kwargs={'name': 'Download table'})

download_file_box = pn.WidgetBox('## Download table', filename, button_download)


tab2 = pn.Column(
    pn.layout.Divider(),
    pn.Row(configuration_box),
    filter_table,
    pn.Row(download_file_box),
    pn.layout.Divider(),
)


#**********************************
# Tab 3: getting numerical results
# *********************************
button_summary_results = pn.widgets.Button(name="Get Summary of Results", button_type="primary", sizing_mode='stretch_width')
button_display_charts = pn.widgets.Button(name="Get plots", button_type="primary", sizing_mode='stretch_width')

summary_results = pn.bind(db.summary_results)
pie_chart_true_labels = pn.bind(db.display_pie_charts, label="true")
pie_chart_predicted = pn.bind(db.display_pie_charts, label="predicted")


def result(clicked):
    if clicked:
        return summary_results()

def result2(clicked):
    if clicked:
        return pie_chart_true_labels()

def result3(clicked):
    if clicked:
        return pie_chart_predicted()

df_results = pn.bind(result, button_summary_results)
display_pie_chart_true = pn.Column(pn.bind(result2, button_display_charts), name="True Labels")
display_pie_chart_predicted = pn.Column(pn.bind(result3, button_display_charts), name="Predicted Labels")

# Create the Accordion with named sections
accordion_pie_charts = pn.Accordion(
    ("True Labels", display_pie_chart_true),
    ("Predicted Labels", display_pie_chart_predicted)
)

tab3 = pn.Column(
    pn.layout.Divider(),
    pn.Row(button_summary_results),
    df_results,
    pn.Row(button_display_charts),
    pn.Row(accordion_pie_charts),
    pn.layout.Divider(),
)


#****************************************************************************
# Tab 4: create visualizations of the original graph and the predicted graph
#****************************************************************************
button_create_original_graph = pn.widgets.Button(name="Display original graph", button_type="primary", sizing_mode='stretch_width')
button_create_predicted_graph = pn.widgets.Button(name="Display predicted graph", button_type="primary", sizing_mode='stretch_width')
button_graphs = pn.widgets.Button(name="Visualize graphs", button_type="primary", sizing_mode='stretch_width')

create_original_graph = pn.bind(db.display_graph, label="true")
create_predicted_graph = pn.bind(db.display_graph, label="predicted")

def result(clicked):
    if clicked:
        return create_original_graph()

def result2(clicked):
    if clicked:
        return create_predicted_graph()

original_graph = pn.Column(pn.bind(result, button_graphs), name="Original graph")
predicted_graph = pn.Column(pn.bind(result2, button_graphs), name="Predicted graph")

accordion_graphs = pn.Accordion(original_graph, predicted_graph)

tab4 = pn.Column(
    pn.layout.Divider(),
    pn.Row(button_graphs),
    pn.Row(accordion_graphs),
    pn.layout.Divider(),
)



# Unificate all the characteristics into a panel
panel_graph = pn.Column(pn.Row(pn.pane.Markdown('# Graph Analysis')),
                        pn.Tabs(('Load file', tab1), ('Predictions', tab2), ('Results', tab3),('Visualizations', tab4)))
panel_graph

BokehModel(combine_events=True, render_bundle={'docs_json': {'b403cce6-9bbe-49fb-af1f-a67973766e3f': {'version…

BokehModel(combine_events=True, render_bundle={'docs_json': {'4f969901-ff6b-484b-ad06-022514a0a88e': {'version…

In [11]:
# Instantiate the template with widgets displayed in the sidebar
template = pn.template.MaterialTemplate(
    title='ANOMALY DETECTION IN GRAPHS',
    sidebar=["Upload file", "Predictions", "Results", "Visualizations"],
)
# Append a layout to the main area, to demonstrate the list-like API
template.main.append(
    panel_graph
)

template.show()

Launching server at http://localhost:59313
